# Coleta e preparação dos dados

## Dados do Twitter

A etapa inicial do projeto é a obtenção dos dados do Twitter, faremos a coleta dos dados, para isso utilizaremos de duas fontes, um conjunto de dados presente no Kraggle e um conjunto de dados obtido através da API do Twitter.

In [2]:
import pandas as pd
#import tweepy as tw
import pickle as pkl
import getpass
import time
from tqdm import tqdm

### Dados do Kraggle
O conjunto de dados do Kraggle _Jair Bolsonaro Twitter Data_ apresenta uma coleção de 3 mil tweets do Bolsonaro.

In [3]:
kraggle_data = pd.read_csv("..//data//tweets//bolsonaro_tweets.csv")
kraggle_data.date = pd.to_datetime(kraggle_data.date)
kraggle_data.head()

,date,text,likes,retweets,link
0,2020-08-01,- Ministro @tarcisiogdf e os 160 anos da Infra...,880,163,https://twitter.com/jairbolsonaro/status/12895...
1,2020-08-01,"- Coletiva com o prefeito de Bagé/RS, Divaldo ...",336,51,https://twitter.com/jairbolsonaro/status/12895...
2,2020-08-01,@rsallesmma 👍🏻,442,39,https://twitter.com/jairbolsonaro/status/12893...
3,2020-08-01,@Marceloscf2 @tarcisiogdf Seguimos! 🤝,164,22,https://twitter.com/jairbolsonaro/status/12893...
4,2020-08-01,@ItamaratyGovBr @USAID 👍🏻,223,24,https://twitter.com/jairbolsonaro/status/12893...


In [6]:
print("O período de " +  str(len(kraggle_data))+" tweets é de " +  str(kraggle_data.date.min())[0:10] +" até " +  str(kraggle_data.date.max())[0:10] + ".")

O período de 9351 tweets é de 2010-04-01 até 2020-08-01.


### Twitter API
Nós também podemos obter dados sobre Jair Bolsonaro, Flávio Bolsonaro, Carlos Bolsonaro e Eduardo Bolsonaro da API do Twitter usando Tweepy. No entanto, o acesso é permitido apenas para os últimos 3200 tweets, dessa forma, não é o histórico completo de tweets.

In [3]:
consumer_key = getpass.getpass()

········


In [4]:
consumer_secret = getpass.getpass()

········


In [5]:
access_token = getpass.getpass()

········


In [6]:
access_token_secret = getpass.getpass()

········


In [7]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [20]:
#requesting data from twitter
twitter_data = {}
for user in ['jairbolsonaro', 'bolsonarosp', 'flaviobolsonaro', 'carlosbolsonaro']:
    twitter_data[user] = [tweet for tweet in tw.Cursor(api.user_timeline,id=user, tweet_mode ="extended").items()]

In [22]:
#saving original data
with open("..//data//tweets//brute_scrapping,pkl", "wb+") as f:
    pkl.dump(twitter_data, f)

In [36]:
#script to create a dataframe from the data
date = []
text = []
user_name = []
place = []
retweets = []
favorites = []
for user in ['jairbolsonaro', 'bolsonarosp', 'flaviobolsonaro', 'carlosbolsonaro']:
    for tweet in twitter_data[user]:
        date.append(tweet._json['created_at'])
        text.append(tweet._json['full_text'])
        user_name.append(user)
        place.append(tweet._json['place'])
        retweets.append(tweet._json['retweet_count'])
        favorites.append(tweet._json['favorite_count'])
API_data = pd.DataFrame({'name': user_name, 'text': text, 'date': pd.to_datetime(date), 
                         'place': place, 'retweets': retweets, 'favorites':favorites})
API_data.head()

,name,text,date,place,retweets,favorites
0,jairbolsonaro,"-Edifício Joelma/SP, 1974.\n\n-Sgt CASSANIGA s...",2020-07-27 20:51:13+00:00,None,3154,16202
1,jairbolsonaro,- Água para quem tem sede.\n- Liberdade para u...,2020-07-27 11:10:36+00:00,None,8101,37357
2,jairbolsonaro,"@tarcisiogdf @MInfraestrutura 🤝🇧🇷, Ministro!",2020-07-26 20:18:19+00:00,None,1074,16840
3,jairbolsonaro,2- @MinEconomia @MinCidadania @onyxlorenzoni @...,2020-07-26 15:40:39+00:00,None,1337,6383
4,jairbolsonaro,1- Acompanhe as redes sociais! @secomvc @fabio...,2020-07-26 15:39:47+00:00,None,3287,14836


In [51]:
API_data.to_csv('..API_data.csv', sep = "~")

## Dados econômicos

Nós vamos coletar os dados do BACEN (Banco Central do Brasil) usando sua API. Nós vamos utilizar as séries temporais de alguns importantes indicadores econômicos como GDP, IPCA, NPCC. O código presente neste notebook é baseado no vídeo de __Código Quant - Finanças Quantitativas__: [COMO ACESSAR A BASE DE DADOS DO BANCO CENTRAL DO BRASIL COM PYTHON | Python para Investimentos #10](https://www.youtube.com/watch?v=7rFsu48oBn8).

In [4]:
def query_bc(code):
    url = 'http://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados?formato=json'.format(code)
    df = pd.read_json(url)
    df['data'] = pd.to_datetime(df['data'], dayfirst=True)
    df.set_index('data', inplace=True)
    return df

As séries temporais escolhidas para compor o nosso conjunto de dados estão listadas abaixo, tabmém está listado sua unidade e o instituto que publica o indicador:

- IPCA : Índice nacional de preços ao consumidor-amplo, Var. % mensal, (IBGE). Este índice mede a variação de preços de mercado para o consumidor final e é utilizado para medir a inflação.
- INPC: Índice nacional de preços ao consumidor, Var. % mensal, (IBGE). Este índice mede a variação de preços da cesta de consumo da população assalariada com mais baixo rendimento.
- IGP-M : Índice geral de preços do mercado, Var. % mensal, (FGV). Este índice mede a variação de preços através de outros índices, do IPCA, INCC, e IPC.
- Meta SELIC : Taxa básica de juros definida pelo Copom, % a.a., (Copom).
- CDI : Taxa de juros, % a.d., (Cetip). Taxa de juros para empréstimo entre bancos.
- PIB : Produto Interno Bruto mensal, R\$ (milhões), (BCB-Depec). 
- Dólar: Taxa de câmbio - Dólar americano (venda), u.m.c/US\$, (Sisbacen PTAX800).
- Dívida pública: Dívida líquida do Setor Público (%PIB) - Total, %, (BSB - DSTAT).
- Reserva Internacional: Reserva internacional total diária, US\$ (milhões), (BCB - DSTAT). 
- Índice de emprego formal, (Mtb).
- PNADC: Taxa de desocupação, %, (IBGE). 
- Índice de confiança do consumidor, (Fecomercio).

In [9]:
index_dict = {"ipca": 433,  "igpm": 189, "inpc": 188, "selic_meta": 432,
              "international_reserve": 13621, "pnad ": 24369,  "cdi" : 12,
              "gdp": 4385, "dollar": 1, "employment": 25239, "gov_debt": 4503,
              "consumer_confidence": 4393, "ibovespa": 7832}
series_list = []
for (key, value) in tqdm(index_dict.items()):
    serie = query_bc(value)
    serie.to_csv("..\\data\\economic_data\\"+key+".csv", sep = ";")
    series_list.append(serie)
    time.sleep(2)

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:33<00:00,  2.59s/it]


Vamos gerar um _dataframe_ final incluindo todas as séries temporais econômicas, note que existirão células vazias pois os indicadores econômicos apresentam frequências diferentes, podem ser diários, mensais, trimestrais ou anuais, e também os indicadores começaram a ser utilizados em momentos distintos na história.

In [12]:
from functools import reduce
economic_time_series = reduce(lambda x, y: pd.merge(x, y, left_index = True, right_index = True, how = 'outer'), series_list)
economic_time_series.columns = ["ipca", "igpm", "inpc", "selic_meta", "international_reserve",
                                      "pnad", "cdi", "gdp", "dollar", "employment", "gov_debt", "consumer_confidence", "ibovespa"]
economic_time_series.to_csv("..\\data\\economic_data\\economic_time_series.csv", sep = ";")